<!-- dom:TITLE: Programmation Python  pour les mathématiques -->
# Python Programming for Mathematics
<!-- dom:AUTHOR: Julien Guillod at [Sorbonne Université](http://www.sorbonne-universite.fr/), -->
<!-- Author: -->  
**Julien Guillod**, [Sorbonne Université](http://www.sorbonne-universite.fr/),
Licensed <a href="https://creativecommons.org/licenses/by-nc-nd/4.0/">CC BY-NC-ND</a>


All chapters are available in
[HTML](https://python.guillod.org//) and [PDF](https://python.guillod.org//python.pdf).
This notebook is also executed on [mybinder](https://mybinder.org/v2/gh/juguillod/python/master?filepath=chap06.ipynb).






# 6 Algebra
<div id="ch:algebre"></div>

First, we study a direct algorithm to solve a linear equation system, then another iterative method to find the eigenvector associated with the biggest eigenvalue of a matrix. Finally, we look into permutation groups.

**This chapter covers:**

* direct method of solving a linear system (LU decomposition)

* algorithm *inplace*

* iterative method (power iteration)

* permutations groups

* orbits et stabilisers





<!-- --- begin exercise --- -->

# Exercise 6.1: LU decomposition
<div id="exer:algebre-LU"></div>

The LU decomposition is a method of decomposing a matrix $A$ of size $n \times n$ into the form $A=LU$ where $L$ is a lower triangular matrix with ones on its diagonal, and $U$ an upper triangular matrix. Once decomposed, the $A=LU$ matrix can be easily used to solve the linear problem $A\boldsymbol{x} = \boldsymbol{b}$ by solving first for $L\boldsymbol{y} = \boldsymbol{b}$ and then $U\boldsymbol{x} = \boldsymbol{y}$. One advantage of the LU decomposition with the Gaussian algorithm for example, is that once such decomposition is known, it is then possible to solve the linear system quickly with different second member systems.

Since $l_{ik}=0$ for $k>i$, we have:

$$
a_{ij} = \sum_{k=1}^n l_{ik}u_{kj} = l_{ii}u_{ij} + \sum_{k=1}^{i-1} l_{ik}u_{kj} \,,
$$

and since $l_{ii}=1$:

$$
u_{ij} = a_{ij} - \sum_{k=1}^{i-1} l_{ik}u_{kj} \,.
$$

On the other hand, since $u_{kj}=0$ for $k>j$, then:

$$
a_{ij} = \sum_{k=1}^n l_{ik}u_{kj} = l_{ij}u_{jj} + \sum_{k=1}^{j-1} l_{ik}u_{kj} \,,
$$

and so

$$
l_{ij} = \frac{1}{u_{jj}} \left( a_{ij} - \sum_{k=1}^{j-1} l_{ik}u_{kj} \right) \,.
$$

Hence, if the $(i-1)$ first lines of $U$ and $(i-1)$ first columns of $L$ are known, it is then possible to determine the $i$-th line of $U$:

$$
u_{ij} = a_{ij} - \sum_{k=1}^{i-1} l_{ik}u_{kj} \,,
$$

then the $i$-th column of $L$:

$$
l_{ji} = \frac{1}{u_{ii}} \left( a_{ji} - \sum_{k=1}^{i-1} l_{jk}u_{ki} \right) \,.
$$

**a)**
Write a function `LU(A)` returning the results of the LU decomposition of a matrix.

In [1]:
import numpy as np

In [2]:
def LU(A):
    n = len(A)
    L = np.zeros((n,n))
    U = np.zeros((n,n))
    
    L[:,0] = A[:,0]/A[0,0]
    U[0,:] = A[0,:]
    for i in range(1,n):
        for j in range(1,n):
            sumU = 0
            sumL = 0
            for k in range(i):
                sumU += L[i,k]*U[k,j]
                sumL += L[j,k]*U[k,i]
            if j == 1:
                U[i,i] = A[i,i] - sumU
            U[i,j] = A[i,j] - sumU
            L[j,i] = 1/U[i,i] * (A[j,i] - sumL)
    return L, U

A = np.array([
    [1,1,1],
    [4,3,-1],
    [3,5,3]
], dtype='float')

L, U = LU(A)

print(L)
print(U)

[[ 1.  0.  0.]
 [ 4.  1. -0.]
 [ 3. -2.  1.]]
[[  1.   1.   1.]
 [  0.  -1.  -5.]
 [  0.   0. -10.]]



**b)**
Write a function `LU_inplace(A)` which does not create two new matrices for $L$ and $U$ but instead modify $A$ so that its lower triangular part (without the diagonal) corresponds to $L$, and its upper part (with the diagonal) corresponds to $U$.

In [3]:
def LU_inplace(A):
    L, U = LU(A)
    n = A.shape[0]
    for i in range(n):
        L[i,i:] = U[i,i:]
    A[:,:] = L

LU_inplace(A)
A

array([[  1.,   1.,   1.],
       [  4.,  -1.,  -5.],
       [  3.,  -2., -10.]])


**c)**
Given the LU decomposition of a matrix $A$, write a function `solve(L,U,b)` solving the linear equation system $A\boldsymbol{x} =  \boldsymbol{b}$.

In [4]:
def solve(L, U, b):
    # Ly = b and Ux = y
    n = len(L)
    x = np.zeros(n)
    y = np.zeros(n)
    
    y[0] = b[0]
    for i in range(1,n):
        s = 0
        for j in range(i):
            s += L[i,j]*y[j]
        y[i] = b[i] - s
    
    x[n-1] = y[n-1]/U[n-1,n-1]
    
    for i in range(1,n):
        s = 0
        for j in range(i+1):
            s += U[n-1-i,n-1-j]*x[n-1-j]
        x[n-1-i] = (y[n-1-i] - s) / U[n-1-i,n-1-i]
    return x

A = np.array([
    [1,1,1],
    [4,3,-1],
    [3,5,3]
], dtype='float')

L, U = LU(A)

b = np.array([1,6,4])

solve(L, U, b)

array([ 1. ,  0.5, -0.5])


**d)**
Using the LU decomposition of the matrix $A$, write a function `det(A)` returning its determinant.

In [5]:
def det(A):
    # det(A) = det(L.U) = det(L) * det(U) = det(U) since det(L) = 1
    L, U = LU(A)
    return np.product(np.diagonal(U))

det(A)

10.0

In [6]:
# Numpy's built-in function to find the determinant of the matrix A
np.linalg.det(A)

10.000000000000002



<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

# Exercise 6.2: Power iteration method
<div id="exer:algebre-power"></div>

The goal of this exercise is to determine the eigenvector of a matrix associated with the biggest eigenvalue. Provided a matrix $A$ of size $n\times n$ and a vector $\boldsymbol{x}_0\in\mathbb{R}^n$, the sequence of vectors $(\boldsymbol{x}_k)_ {k\in\mathbb{N}}$ is defined by:

$$
\boldsymbol{x}_{k+1} = \frac{A\boldsymbol{x}_k}{\Vert A\boldsymbol{x}_k\Vert} \,.
$$

By supposing that the matrix $A$ is diagonalisable with distinct eigenvalues, it is relatively easy to show that the sequence $(\boldsymbol{x}_k)_ {k\in\mathbb{N}}$ converges to the eigenvector associated with the biggest eigenvalue of $A$.

<!-- --- begin hint in exercise --- -->

**Indication:**
The first step is to realise that

$$
\boldsymbol{x}_k = \frac{A^2 \boldsymbol{x}_{k-2}}{\Vert A^2 \boldsymbol{x}_{k-2}\Vert} = \dots = \frac{A^k \boldsymbol{x}_0}{\Vert A^k \boldsymbol{x}_0\Vert} \,.
$$

Since $A$ is diagonalisable, let ($\boldsymbol{v}_1, \boldsymbol{v}_2, \dots, \boldsymbol{v}_n)$ be a base of eigenvalues of $A$ associated with the eigenvalues $\lambda_1, \lambda_2, \dots, \lambda_n$. Without losing generality, we suppose that $\lambda_1$ is the biggest eigenvalue. The vector $\boldsymbol{x}_0$ is decomposed within the base:

$$
\boldsymbol{x}_0 = \sum_{i=1}^n c_i \boldsymbol{v}_i \,,
$$

then supposing that $c_1\neq0$:

$$
A^k \boldsymbol{x}_0 = \sum_{i=1}^n c_i \lambda_i^k \boldsymbol{v}_i
= c_1 \lambda_1^k \left(\boldsymbol{v}_1 + \sum_{i=2}^n \frac{c_i}{c_1} \left(\frac{\lambda_i}{\lambda_1}\right)^k \boldsymbol{v}_i \right) .
$$

Since $\lambda_1 > \lambda_i$ for $i\leq2$, then:

$$
\lim_{k\to\infty}\left(\boldsymbol{v}_1 + \sum_{i=2}^n \frac{c_i}{c_1} \left(\frac{\lambda_i}{\lambda_1}\right)^k \boldsymbol{v}_i \right) = \boldsymbol{v}_1 \,,
$$

and thus

$$
\lim_{k\to\infty}\boldsymbol{x}_k = \lim_{k\to\infty}\frac{A^k \boldsymbol{x}_0}{\Vert A^k \boldsymbol{x}_0\Vert} = \frac{\boldsymbol{v}_1}{\Vert\boldsymbol{v}_1\Vert} \,.
$$

When $c_1\neq0$, the sequence $(\boldsymbol{x}_k)_ {k\in\mathbb{N}}$ converges to the normalised eigenvector associated with the biggest eigenvalue.

<!-- --- end hint in exercise --- -->


**a)**
Define a function `power(A,x0,k)` returning $\boldsymbol{x}_k$. With this function, determine the biggest eigenvalue of the matrix

$$
A = \begin{pmatrix}0.5 & 0.5\\ 
0.2 & 0.8
\end{pmatrix} \,.
$$

<!-- --- begin answer of exercise --- -->
**Answer:**
The eigenvector associated with the biggest eigenvalue is $\pm(0.70710678, 0.70710678)$

<!-- --- end answer of exercise --- -->



In [7]:
from numpy import linalg as LA

def power(A, x0, k):
    for i in range(1,k):
        dot_product = np.dot(A, x0)
        xi = dot_product / np.linalg.norm(dot_product)
        x0 = xi
    return xi
        

A = np.array([[0.5,0.5],[0.2,0.8]])
x0 = np.array([[1,2]]).T
k = 10

power(A, x0, k)

array([[0.70710272],
       [0.70711084]])


**b)**
Determine the eigenvalue associated with the above vector.

In [9]:
v = power(A, x0, k)

# Since the dot product of A and v equals lambda (the associated eigenvalue) times v, we just have to devide
# the first cell of A*v by the first cell of v to find lambda

Av = np.dot(A, v)
Av[0,0]/v[0,0]

1.0000057409220828


**c)**
Write a function calculating automatically the biggest eigenvalue and the associated eigenvector of a square matrix.

In [10]:
def biggest_eigen(A):
    x0 = np.random.randint(-10, 10, size=(len(A),1))
    v = power(A, x0, 30)
    Av = np.dot(A[0:], v)
    val = Av[0,0]/v[0,0]
    return val, v
    
B = np.array([[3,6,7],
              [3,3,7],
              [5,6,5]])
biggest_eigen(B)

(15.0,
 array([[-0.6092077 ],
        [-0.50767308],
        [-0.6092077 ]]))

**d)**
Look into the documentation of Numpy and find a function that calculates the eigenvectors and eigenvalues of a matrix

In [11]:
# numpy.linalg.eig()

val, v = np.linalg.eig(B)
print(val)
print(v)

[15.         -2.00000003 -1.99999997]
[[ 0.6092077   0.40824827 -0.40824831]
 [ 0.50767308 -0.81649658  0.81649658]
 [ 0.6092077   0.4082483  -0.40824828]]



**e)**
Compare the speed of the above code and the Numpy's version with different values of $n=10,100,1000$.

In [12]:
import time

for n in [10,100,1000]:
    print("n =",n)
    A = np.random.random((n,n))
    # Home-made function
    start = time.time()
    exe = biggest_eigen(A)
    print("My func took:", time.time() - start)
    
    # Numpy function
    start = time.time()
    exe = LA.eig(A)
    print("NP func took:", time.time() - start)
    
    print()

n = 10
My func took: 0.0008299350738525391
NP func took: 0.0002391338348388672

n = 100
My func took: 0.0021638870239257812
NP func took: 0.006815910339355469

n = 1000
My func took: 0.007632732391357422
NP func took: 0.9560551643371582




<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

# Exercise 6.3: Permutation groups
<div id="exer:algebre-permutations"></div>

Le goal is to study permutation groups by developping some algorithms to characterise them. A permutation group $G$ is generated by a certain number of permutations: $G = \langle g_1, g_2,\dots,g_k \rangle$. A permutation

$$
g = \begin{pmatrix}1 & 2 & 3 & 4\\ 
4 & 1 & 2 & 3
\end{pmatrix} \,,
$$

can be represented in Python by a tuple `g = (0, 4, 1, 2, 3)`. The zero is added to assure correct indexing in Python and simplify further implementations. It means simply that the number `0` goes to `0`. 


**a)**
Write a function `product(g1,g2)` calculating the product of two permutations.

In [5]:
def product(g1,g2):
    diff_len = abs(len(g1) - len(g2))
    n = len(g2)
    if diff_len != 0:
        mini = min([g1,g2], key=len)
        maxi = max([g1,g2], key=len)
        g1 = tuple(list(mini) + [i + len(mini) for i in range(diff_len)])
        g2 = maxi
        n = len(g2)
    produit = [0]
    for i in range(1,n):
        produit.append(g1[g2[i]])
    return tuple(produit)

g1 = (0, 3, 1, 5, 4, 2)
g2 = (0, 6, 2, 4, 1, 5, 3)
product(g1,g2)

(0, 6, 1, 4, 3, 2, 5)


**b)**
Write a function `inverse(g)` calculating the inversion of a permutation.

In [14]:
def inverse(g):
    n = len(g)
    inv = [0]
    for i in range(1,n):
        inv.append(g.index(i))
    return tuple(inv)

g = (0, 3, 1, 4, 2, 5)
inverse(g)

(0, 2, 4, 1, 3, 5)


**c)**
Write a function returning the decomposition of a permutation under the cyclic notation, represented by a list of tuples.

In [15]:
def cyclic_notation(g):
    group = list(g)
    results = list()
    cycle = list()
    i = 1
    while len(group) > 1:
        cycle.append(i)
        group.remove(g[i])
        if g[i] == cycle[0]:
            if len(cycle) != 1:
                results.append(tuple(cycle))
            cycle = list()
            if len(group) == 1:
                break
            i = group[1]
        else:
            i = g[i]    
    return results

g = (0, 4, 5, 3, 7, 2, 6, 8, 1)
cyclic_notation(g)

[(1, 4, 7, 8), (5, 2)]


**d)**
Write a function returning the permutation corresponding to a cyclic notation, meaning the inversion of the above function.

In [16]:
def to_permu(cycles):
    length = max(max(cycles, key=max))
    results = [i for i in range(length+1)]
    for cycle in cycles:
        for i, x in enumerate(cycle):
            if i < len(cycle)-1:
                results[x] = cycle[i+1]
            else:
                results[x] = cycle[0]
    return tuple(results)

cyc = [(1, 4, 7, 8), (5, 2)]
to_permu(cyc)

(0, 4, 5, 3, 7, 2, 6, 8, 1)


**e)**
<span style="color:red">!</span> In Python a group $G = \langle g_1,g_2,\dots,g_k \rangle$ generated by permutations, can be represented by a list of permutations `G = [g1,g2,...,gk]`. Write a fonction `orbit(G,x)` returning the orbit of the element $x \in G$:

$$
O_x = Gx = \{gx, g \in G\} \,.
$$

<!-- --- begin hint in exercise --- -->

**Indication:**
Do not calculate the set of elements of the groups, doing so would make a very large group. Instead, construct a list $(X^0,X^1,\dots,X^N)$ of disjoint sets defined recursively by $X^0 = \{x\}$ and $X^n$ as a set of new elements under the form $g_i y$ with $1 \leq i \leq k$ and $y\in X^{n-1}$:

$$
X^{n}=\left(\bigcup_{i=1}^{k}g_{i}X^{n-1}\right)\setminus\left(\bigcup_{i=1}^{n-1}X^{i}\right) \,.
$$

<!-- --- end hint in exercise --- -->

In [26]:
from itertools import permutations
from math import factorial

k = 10

# To simplify initialisation, we make G a symmetrical group of size k=10, which is already quite large, with has 10! = 3,628,800 elements
G = [tuple([0] + list(p)) for p in permutations([i for i in range(1, k+1)])]

len(G) == factorial(k)  # Check whether our group is well defined

True

In [21]:
import time
import random

def orbit(G, x):
    Ox = set()
    for g in G:
        Ox.add(product(g, x))
    return Ox

x = random.choice(G)  # Choose randomly a permutation among our group G
# G = [(0, 1, 2, 3), (0, 1, 3, 2)]

start = time.time()
result = orbit(G, x)
print("Calculation time:", time.time() - start)

Calculation time: 8.258555889129639



**f)**
<span style="color:red">!!</span>Define a function `stabilizer(G,x)` returning the stabiliser of $x \in G$:

$$
G_x = \{g \in G : g x = x \} \,,
$$

In [29]:
def stabilizer(G, x):
    Gx = {g for g in G if product(g,x) == x}
    return Gx

stabilizer(G, x)

{(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10)}

In [30]:
# As expected, the identity permutation !

under the form of a set of generators.

<!-- --- begin hint in exercise --- -->

**Indication:**
Use the theory of Schreier.

<!-- --- end hint in exercise --- -->




<!-- --- end exercise --- -->